# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [142]:
import pandas as pd

df = pd.read_csv('churn_data_week3_prepared.csv', index_col='customerID')
df
df.drop('percentage_rate_charge', axis=1, inplace=True)

In [143]:
df1 = df.copy()
df1

tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
customerID                                                                  
7590-VHVEG       1             0         0              0           29.85   
5575-GNVDE      34             1         1              1           56.95   
3668-QPYBK       2             1         0              1           53.85   
7795-CFOCW      45             0         1              2           42.30   
9237-HQITU       2             1         0              0           70.70   
...            ...           ...       ...            ...             ...   
6840-RESVB      24             1         1              1           84.80   
2234-XADUH      72             1         1              3          103.20   
4801-JZAZL      11             0         0              0           29.60   
8361-LTMKD       4             1         0              1           74.40   
3186-AJIEK      66             1         2              2          105.65   

            TotalCharges  Churn  
customerID                       
7590-VHVEG         29.85      0  
5575-GNVDE       1889.50      0  
3668-QPYBK        108.15      1  
7795-CFOCW       1840.75      0  
9237-HQITU        151.65      1  
...                  ...    ...  
6840-RESVB       1990.50      0  
2234-XADUH       7362.90      0  
4801-JZAZL        346.45      0  
8361-LTMKD        306.60      1  
3186-AJIEK       6844.50      0  

[7032 rows x 7 columns]

# AutoML with pycaret

In [144]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [145]:
?setup

setup our autoML

In [146]:
automl = setup(df1, target='Churn', preprocess=False)

In [147]:
automl[6]

False

In [148]:
best_model = compare_models()

In [149]:
best_model

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=2680)

In [150]:
df1.iloc[-2:-1].shape

(1, 7)

In [151]:
predict_model(best_model, df1.iloc[-2:-1])

tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
customerID                                                                  
8361-LTMKD       4             1         0              1            74.4   

            TotalCharges  Churn  Label   Score  
customerID                                      
8361-LTMKD         306.6      1      1  0.5018

# Saving and loading our model

In [152]:
save_model(best_model, 'LDA')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ['trained_model',
                  AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                                     learning_rate=1.0, n_estimators=50,
                                     random_state=2680)]],
          verbose=False),
 'LDA.pkl')

In [153]:
import pickle

with open('LDA_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [154]:
import pickle

with open('LDA_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [155]:
new_data = df1.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1])

In [156]:
loaded_lda = load_model('LDA')

Transformation Pipeline and Model Successfully Loaded


In [157]:
predict_model(loaded_lda, new_data)

tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
customerID                                                                  
8361-LTMKD       4             1         0              1            74.4   

            TotalCharges  Label   Score  
customerID                               
8361-LTMKD         306.6      1  0.5018

# Making a Python module to make predictions

In [158]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads Churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('LDA')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No Churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [159]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC       Churn
1452-KNGVK    No Churn
6723-OKKJM    No Churn
7832-POPKP    No Churn
6348-TACGU    No Churn
Name: Churn_prediction, dtype: object


# Summary

In this DS Automation, We have loaded our prepared data and have dropped the percentage rate charge's variable, so it would fit the model we created after. We made a copy of our data, and so we could run our lab with the copy instead of the original. Then, we have used pycaret to find an ML algorithm that perform best on our data. We save the model on our disk. We have created a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe (We have used the new data, new_churn_data.csv).

Finally we have tested our Python module and function with the new data, new_churn_data.csv